In preparation to this script we manually matched transactions and units. <br>
This script cleans the resulting datasets, prepares them for the analysis and merges them into one final dataset. <br>
In the end we will get one dataset with only unique units and one with unit-transactions.<br>
Each of the two will contain information on finance and on other involved actors.

# 0 Import libraries

In [1]:
import pandas as pd
import numpy as np

# 1 Import and prepare the manually matched datasets

We manually matched the financing data from urgewald and GEM with units. Both are imported and merged in the following. The resulting dataset is prepared for the final merging with the Platts data.

## 1.1 Import datasets

In [2]:
pz = '2021_01' #date of GEM plant tracker
fz = '2020_12' #date of GEM finance tracker
uz = '2021_02' #date of urgewald finance data for coal exit list, received per mail

In [75]:
# import WEPP platts data
tz = '2019' #date of newest platts
df_platts_2017 = pd.read_csv('.//tables//2017_df_platts_coal.csv', sep=';', index_col=0, encoding='latin-1')
df_platts_2019 = pd.read_csv('.//tables//2019_df_platts_coal.csv', sep=';', index_col=0, encoding='latin-1')

df_platts_abbrev = pd.read_csv('.//platts_2017_ABBREV.csv', sep=';', encoding='latin-1')

In [4]:
#import the dataset where public finance data from the Global Coal Plant Finance Tracker from Global Energy Monitor
#is manually linked to units

folder = './/'
filename = 'plants_finance_merge_Leon_GCPT_'+pz+'_GCFT_'+fz+'_deflation_20210224.csv' 
df_gem_gem = pd.read_csv(folder+filename ,sep=';', encoding='latin-1', decimal=',', thousands='.', index_col=0)

# for df_gem_gem: sometimes values are nan, replace with 0:
for x in ['Amount (in USD)','MW']:
    df_gem_gem[x] = df_gem_gem[x].fillna(0)#.astype(int)

#sometimes no Phase ID stated:
n = 1
for x in df_gem_gem.index:
    if type(df_gem_gem.loc[x, 'Phase ID']) != str:
        df_gem_gem.loc[x, 'Phase ID'] = 'PHx'+str(n)
        n+=1
        
#include a column with the origin of the data:
df_gem_gem['data'] = 'GCPFT_GEM'

# add 'GCPFT_GEM' to Transaction Nr to differentiate from other dataset
for x in df_gem_gem.index:
    df_gem_gem.loc[x, 'Transaction Nr'] = str(df_gem_gem.loc[x, 'Transaction Nr'])+'_GCPFT_GEM'

#rename a column, so it matches the column from urgewald
df_gem_gem.rename(columns={'Status.plt':'Status'}, inplace=True)

In [5]:
#import the dataset where the mostly private finance data of the Global Coal Exit list from urgewald
#is manually linked to unit
folder = './/'
filename = 'plants_finance_merge_Leon_GCPT_'+pz+'_urg_'+uz+'_deflation_unique_IDs_20210322.csv' 
df_urg_gem_unique_IDs = pd.read_csv(folder+filename ,sep=';', encoding='latin-1', decimal=',', thousands='.', index_col=0)

# for df_urg_gem_unique_IDs: sometimes values are nan, replace with 0:
for x in ['Amount (in USD 2020)', 'Per Investor Value (in mln US$)']:
    df_urg_gem_unique_IDs[x] = df_urg_gem_unique_IDs[x].fillna(0)#.astype(int)

In [6]:
# in the manually matched datasets, every transaction ID is only present once.
# in the original dataset most IDs are stated multiple times, because of differing banks (syndicate loans)
# match the individual unit-transactions with the whole dataset:
df_urg_all = pd.read_csv('.//urgewald_Financing_of_GCEL_2020_creditor_nogreen+inscope+project+noIFI.csv' ,\
                     sep=';', encoding='latin-1', decimal=',', thousands='.')
df_urg_gem = df_urg_gem_unique_IDs.copy()
for x in df_urg_gem_unique_IDs.index:
    for y in df_urg_all.index:
        if ((df_urg_gem_unique_IDs.loc[x, 'Deal Number / Deal ID'] == df_urg_all.loc[y, 'Deal Number / Deal ID']) \
            and (df_urg_gem_unique_IDs.loc[x, 'Investor Parent'] != df_urg_all.loc[y, 'Investor Parent'])):
            df_urg_gem = df_urg_gem.append(df_urg_gem_unique_IDs.loc[x], ignore_index=True)
            var_last_index = df_urg_gem.index[-1]
            df_urg_gem.loc[var_last_index, 'Investor Parent'] = df_urg_all.loc[y, 'Investor Parent']
            df_urg_gem.loc[var_last_index, 'Investor Parent Country'] = df_urg_all.loc[y, 'Investor Parent Country']

            df_urg_gem.loc[var_last_index, 'Per Investor Value (in mln US$)'] = df_urg_all.loc[y, 'Per Investor Value (in mln US$)']
            df_urg_gem.loc[var_last_index, 'Amount (in USD 2020)'] = df_urg_gem.loc[var_last_index, 'Per Investor Value (in mln US$)']\
            *df_urg_gem.loc[var_last_index, 'Deflation_Factor_2020']*1000000
            
        #sometimes same deal number & bank, but different value!
        elif ((df_urg_gem_unique_IDs.loc[x, 'Deal Number / Deal ID'] == df_urg_all.loc[y, 'Deal Number / Deal ID']) \
            and (df_urg_gem_unique_IDs.loc[x, 'Investor Parent'] == df_urg_all.loc[y, 'Investor Parent']) \
            and (df_urg_gem_unique_IDs.loc[x, 'Per Investor Value (in mln US$)'] != df_urg_all.loc[y, 'Per Investor Value (in mln US$)'])):
            df_urg_gem = df_urg_gem.append(df_urg_gem_unique_IDs.loc[x], ignore_index=True)
            var_last_index = df_urg_gem.index[-1]
            df_urg_gem.loc[var_last_index, 'Investor Parent'] = df_urg_all.loc[y, 'Investor Parent']
            df_urg_gem.loc[var_last_index, 'Investor Parent Country'] = df_urg_all.loc[y, 'Investor Parent Country']

            df_urg_gem.loc[var_last_index, 'Per Investor Value (in mln US$)'] = df_urg_all.loc[y, 'Per Investor Value (in mln US$)']
            df_urg_gem.loc[var_last_index, 'Amount (in USD 2020)'] = df_urg_gem.loc[var_last_index, 'Per Investor Value (in mln US$)']\
            *df_urg_gem.loc[var_last_index, 'Deflation_Factor_2020']*1000000

In [7]:
#adjust df_urg_gem:

#rename some columns so they match df_gem_gem:
df_urg_gem.rename(columns={'Investor Parent':'Institution', 'Investor Parent Country':'Financing Country', \
        'Year.plt':'Year'}, inplace=True)

#include a column with the origin of the data:
df_urg_gem['data'] = 'GCEL_urg'

#rename some columns that are used later so I can match the two financing datasets:
df_urg_gem.rename(columns={'Investor Parent Country':'Financing Country',\
                           'Year.fin':'Year of Close', 'Deal Number / Deal ID':'Phase ID'}, inplace=True)

# add 'GCEL_urg' to Transaction Nr to differentiate from other dataset
for x in df_urg_gem.index:
    df_urg_gem.loc[x, 'Transaction Nr'] = str(df_urg_gem.loc[x, 'Transaction Nr'])+'_GCEL_urg'

In [11]:
# merge the two datasets
df_urggem_gem = df_gem_gem.append(df_urg_gem, sort=False).reset_index(drop=True)

#prepare dict with all three datasets
dict_3_dfs = {'gem_GCFT':df_gem_gem, 'urg_GCEL':df_urg_gem, 'merged':df_urggem_gem}
dict_dfs = {'gem_GCFT':df_gem_gem, 'urg_GCEL':df_urg_gem}

## 1.2 Get ownership of banks

In [15]:
# import sheet where I stated the type of bank (state-owned / private / ECA) for each bank
df_banks = pd.read_csv('.//banks//banks_grouped_20210727_nm.csv' ,sep=';', encoding='latin-1', decimal=',', thousands='.')

In [16]:
# first, check if all banks from df_urggem_gem are in df_banks:
list_banks_not_yet_included =[]
for x in df_urggem_gem.index:
    list_banks = list(set(df_banks['other_name_1'])|set(df_banks['other_name_2'])|\
                      set(df_banks['other_name_3'])|set(df_banks['name']))
    if df_urggem_gem.loc[x, 'Institution'] not in list_banks:
        list_banks_not_yet_included.append(df_urggem_gem.loc[x, 'Institution'])
        
list_banks_not_yet_included = list(set(list_banks_not_yet_included))
len(list_banks_not_yet_included)

0

In [18]:
# if banks are not yet included, match them manually and then import df again!
df_banks_not_yet_included =  pd.DataFrame(data={'banks':list_banks_not_yet_included})
df_banks_not_yet_included.to_csv('.//banks//banks_not_yet_included_20210322.csv' ,sep=';', encoding='latin-1', decimal=',')

In [17]:
# change name of banks, if there are different names for the same bank:
print('no of unique banks before = '+ str(len(set(df_urggem_gem['Institution']))))
for x in df_urggem_gem.index:
    var_bank = df_urggem_gem.loc[x, 'Institution']
    for y in ['other_name_1', 'other_name_2', 'other_name_3']:
        for z in df_banks.index:
            if var_bank == df_banks.loc[z, y]:
                df_urggem_gem.loc[x, 'Institution'] = df_banks.loc[z, 'name']
print('no of unique banks after = '+ str(len(set(df_urggem_gem['Institution']))))

no of unique banks before = 232
no of unique banks after = 215


In [18]:
# include column with type of bank (state-owned / private / ECA)
for x in df_urggem_gem.index:
    var_bank = df_urggem_gem.loc[x, 'Institution']
    df_urggem_gem.loc[x, 'ownership'] = df_banks[df_banks.name==var_bank]['ownership'].reset_index(drop=True)[0]

## 1.3 ECAs as Insurers

ECAs that provide exclusively credit insurance (not loans) are taken as a seperate column (not as banks). <br> The banks from urgewald are not considered as insurers, as they all proide either loans or underwriting of share and bond issuances.

In [20]:
list_ECAs = list(set(df_urggem_gem[df_urggem_gem['ownership']=='ECA']['Institution']))

In [23]:
df_a = df_urggem_gem.copy()
df_no_ECA = df_a[~df_a.Institution.isin(list_ECAs)]

In [24]:
# add insurance as own company in own column:
# create df without the ECAs
df_a = df_urggem_gem.copy()
df_no_ECA = df_a[~df_a.Institution.isin(list_ECAs)]

df_urggem_gem_ECAcol = df_urggem_gem.copy()
for x in ['1','2']:
    df_urggem_gem_ECAcol['Insurer_'+x] = ''
    df_urggem_gem_ECAcol['Insurer_'+x+'_Country'] = ''

for x in df_urggem_gem_ECAcol.index:
    var_insurer = df_urggem_gem_ECAcol.loc[x, 'Institution']
    var_ins_count = df_urggem_gem_ECAcol.loc[x, 'Financing Country']
    if var_insurer in list_ECAs:
        var_ID = df_urggem_gem_ECAcol.loc[x, 'TrackerID']
        if var_ID in set(df_no_ECA['TrackerID']):
            df_b = df_urggem_gem_ECAcol[df_urggem_gem_ECAcol['TrackerID']== var_ID]
            for y in df_b.index:
                if y in df_urggem_gem_ECAcol.index: #might be it is already dropped
                    if df_urggem_gem_ECAcol.loc[y, 'Insurer_1'] == '':
                        df_urggem_gem_ECAcol.loc[y, 'Insurer_1'] = var_insurer
                        df_urggem_gem_ECAcol.loc[y, 'Insurer_1_Country'] = var_ins_count
                    elif df_urggem_gem_ECAcol.loc[y, 'Insurer_1'] != var_insurer:
                        df_urggem_gem_ECAcol.loc[y, 'Insurer_2'] = var_insurer
                        df_urggem_gem_ECAcol.loc[y, 'Insurer_2_Country'] = var_ins_count
            df_urggem_gem_ECAcol.drop(x, inplace=True)
        else:
            print('this ID will disappear: '+ var_ID) #To see, which units will be lost
            df_urggem_gem_ECAcol.drop(x, inplace=True)
df_urggem_gem_ECAcol.reset_index(drop=True, inplace=True)

this ID will disappear: G100883
this ID will disappear: G100884
this ID will disappear: G100885
this ID will disappear: G113103
this ID will disappear: G108242
this ID will disappear: G108241
this ID will disappear: G100485
this ID will disappear: G100484


## 1.4 Drop double countings

Sometimes one bank provides several loans for a unit. Merge these transactions. <br>
This can be either between the two datasets or in either of them.

In [28]:
# create dataframes to only get cross-border flows, i.e. fin country != site country
def foreign_only(df):
    for x in df.index:
        if df.loc[x, 'Financing Country'] == df.loc[x, 'Country']:
            df.drop(x, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return(df)

In [29]:
# check, if the state-owned transactions from urgewald equal the ones from GEM
df_a = df_urggem_gem_ECAcol.copy()
df_a_gem = df_a[df_a['data']=='GCPFT_GEM']
set_a_gem = set(df_a_gem['Institution'])
df_a_urg = df_a[df_a['data']=='GCEL_urg']
df_a_urg = df_a_urg[df_a_urg['ownership']=='state-owned']
df_a_urg = foreign_only(df_a_urg)
set_a_urg = set(df_a_urg['Institution'])

print('no of banks in GCPFT (state-owned & foreign only) by GEM = '+str(len(set_a_gem)))
print('no of state-owned banks in urgewald with foreign fin = '+str(len(set_a_urg)))
set_b = set_a_gem & set_a_urg
print('intersection of the two sets length = ' +str(len(set_b)))
print('the intersecting banks are: '+ str(set_b))
set_c = set_a_urg - set_a_gem
print('the banks from urgewald that are not in the GEM data are: '+ str(set_c))

no of banks in GCPFT (state-owned & foreign only) by GEM = 43
no of state-owned banks in urgewald with foreign fin = 11
intersection of the two sets length = 5
the intersecting banks are: {'Agricultural Bank of China (ABC)', 'Bank of China (BOC)', 'Bank of Communications', 'Industrial and Commercial Bank of China (ICBC)', 'China Construction Bank (CCB)'}
the banks from urgewald that are not in the GEM data are: {'China Minsheng Banking', 'China Zheshang Bank', 'Anbang Insurance Group', 'Investment Corporation of Dubai', 'Abu Dhabi Investment Council', 'National Commercial Bank'}


In [30]:
# check, for those 5 banks, do they have the same transactions (for the same units?)
df_a_gem_5 = df_a_gem[df_a_gem['Institution'].isin(set_b)]
df_a_urg_5 = df_a_urg[df_a_urg['Institution'].isin(set_b)]
print('the 5 banks in gem have '+str(len(df_a_gem_5))+' unit-transactions')
print('the 5 banks in urg have '+str(len(df_a_urg_5))+' unit-transactions')
set_a_gem_5 = set(df_a_gem_5['TrackerID'])
set_a_urg_5 = set(df_a_urg_5['TrackerID'])
print('the 5 banks in gem finance '+str(len(set_a_gem_5))+' units')
print('the 5 banks in urg finance '+str(len(set_a_urg_5))+' units')
set_b_5 = set_a_gem_5 & set_a_urg_5
print('intersection of the two sets length = ' +str(len(set_b_5)))


the 5 banks in gem have 140 unit-transactions
the 5 banks in urg have 138 unit-transactions
the 5 banks in gem finance 83 units
the 5 banks in urg finance 24 units
intersection of the two sets length = 10


In [31]:
# sometimes one bank has several transactions for one unit -->merge
# if the two (or more entries) are from the same source, add the transaction amounts, \
# if not, drop one (as it might be double counting)
list_merged_IDs = []
print('len before = '+str(len(df_urggem_gem_ECAcol)))
print('len before state-owned = '+str(len(df_urggem_gem_ECAcol[df_urggem_gem_ECAcol['ownership']=='state-owned'])))
print('len before private = '+str(len(df_urggem_gem_ECAcol[df_urggem_gem_ECAcol['ownership']=='private']))+'\n')

df_urggem_gem_ECAcol_nodoub = df_urggem_gem_ECAcol.copy()
for x in df_urggem_gem_ECAcol_nodoub.index:
    var_ID = df_urggem_gem_ECAcol_nodoub.loc[x, 'TrackerID']
    var_deal = df_urggem_gem_ECAcol_nodoub.loc[x, 'Phase ID']
    fin_a = df_urggem_gem_ECAcol_nodoub.loc[x, 'Institution']
    var_data = df_urggem_gem_ECAcol_nodoub.loc[x, 'data']
    for y in df_urggem_gem_ECAcol_nodoub.index:
        if x in df_urggem_gem_ECAcol_nodoub.index:
            if x !=y and df_urggem_gem_ECAcol_nodoub.loc[y, 'TrackerID'] == var_ID and \
            df_urggem_gem_ECAcol_nodoub.loc[y, 'Institution'] == fin_a and \
            df_urggem_gem_ECAcol_nodoub.loc[y, 'Phase ID'] == var_deal:
                if df_urggem_gem_ECAcol_nodoub.loc[y, 'data'] == var_data:
                    df_urggem_gem_ECAcol_nodoub.loc[y, 'Amount (in USD 2020)'] += \
                    df_urggem_gem_ECAcol_nodoub.loc[x, 'Amount (in USD 2020)']
                    df_urggem_gem_ECAcol_nodoub.loc[y, 'Per Investor Value (in mln US$)'] += \
                    df_urggem_gem_ECAcol_nodoub.loc[x, 'Per Investor Value (in mln US$)']
                list_merged_IDs.append(df_urggem_gem_ECAcol_nodoub.loc[y, 'TrackerID'])
                df_urggem_gem_ECAcol_nodoub.drop(x, inplace=True)
df_urggem_gem_ECAcol_nodoub.reset_index(drop=True, inplace=True)

print('len afterwards = '+str(len(df_urggem_gem_ECAcol_nodoub)))
print('len afterwards state-owned = '+str(len(df_urggem_gem_ECAcol_nodoub[df_urggem_gem_ECAcol_nodoub['ownership']=='state-owned'])))
print('len afterwards private = '+str(len(df_urggem_gem_ECAcol_nodoub[df_urggem_gem_ECAcol_nodoub['ownership']=='private'])))

len before = 2813
len before state-owned = 1462
len before private = 1351

len afterwards = 2527
len afterwards state-owned = 1329
len afterwards private = 1198


In [32]:
df_urggem_gem_ECAcol_nodoub.to_csv('.//tables//basis//df_urggem_gem_ECAcol_nodoub_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_20210728.csv', \
                       sep=';', encoding='latin-1', decimal=',')

## 1.5 calculate weighted amounts

example for transaction amount: a bank provided 10 mio in loans. We merged this transaction to two units, each of these unit-transactions would now have a transaction amount of 10mio. Thus we split it, i.e. 5mio for each unit-transaction. <br>
example for capacity: if two banks provide finance for a plant with 500MW, the capacity is 500MW each, so double counting. Thus I include a column with the weighted capacity, i.e. 250MW each.

'weighted_transaction_amount': Assigned each transaction-unit-link a weighted_transaction_amount (transaction amount divided by number of units matched to respective transaction)

'mean_weighted_transaction_amount': Assigned each unit a mean_weighted_transaction_amount (total funding of one unit divided by the number of transactions it received). Only count those transactions that are not 0 to calculate the mean. 

'total_funding_amount_estimated': Assigned each unit a total_funding_amount_estimated (sum of weighted_transaction_amounts of all transactions to respective unit plus mean_weighted_transaction_amount for each transaction to respective unit for which amount information was not found). Attention: total_funding_amount_estimated of transactions linked to multiple units was counted multiple times

'weighted_funding_amount_estimated': per unit-transaction: Assigned each transaction-unit-link a weighted_funding_amount_estimated.
weighted_funding_amount_estimated is - if not 0 - the weighted_transaction_amount, otherwise the mean_weighted_transaction_amount for the respective unit.

'funding_share': per unit-transaction: calculate the share of each unit-transaction for a unit. The total sum should equal the no of unique units in the dataframe.

'capacity_weighted_by_funding_share'  Calculate share of capacity (Capacity (MW)) equal to share of finance amount. This makes the actual capacity as of manual matching that was funded/built receiving the transactions)

In [33]:
df_urggem_gem_ECAcol_nodoub_wei = df_urggem_gem_ECAcol_nodoub.copy()

# Create columns
for x in ['weighted_transaction_amount_mio', 'mean_weighted_transaction_amount_mio', 'total_funding_amount_estimated_mio', \
         'weighted_funding_amount_estimated_mio', 'funding_share', 'capacity_weighted_by_funding_share_MW']:
    df_urggem_gem_ECAcol_nodoub_wei[x] = 0

# sometimes nan, replace with 0
for x in ['Amount (in USD 2020)','Capacity (MW)']:
    df_urggem_gem_ECAcol_nodoub_wei[x] = df_urggem_gem_ECAcol_nodoub_wei[x].fillna(0)
df_urggem_gem_ECAcol_nodoub_wei['Amount (in USD 2020)_mio'] = df_urggem_gem_ECAcol_nodoub_wei['Amount (in USD 2020)']/1000000

In [34]:
# 'weighted_transaction_amount'
for x in df_urggem_gem_ECAcol_nodoub_wei.index:
    var_phaseID = df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'Phase ID']
    df_a = df_urggem_gem_ECAcol_nodoub_wei[df_urggem_gem_ECAcol_nodoub_wei['Phase ID']==var_phaseID]
    var_no_of_units = len(set(df_a['TrackerID']))
    var_amount = df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'Amount (in USD 2020)_mio']
    if var_amount == 0:
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'weighted_transaction_amount_mio'] = 0
    else:
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'weighted_transaction_amount_mio'] = \
        var_amount/var_no_of_units

In [35]:
# 'mean_weighted_transaction_amount'
for x in df_urggem_gem_ECAcol_nodoub_wei.index:
    var_TrackerID = df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'TrackerID']
    df_a = df_urggem_gem_ECAcol_nodoub_wei.copy()
    df_b = df_a[df_a['TrackerID']==var_TrackerID]
    var_amount = df_b['weighted_transaction_amount_mio'].sum()
    df_c = df_b[df_b['weighted_transaction_amount_mio']!=0]
    var_no_of_trans_no_0 = len(df_c)
    if var_amount == 0:
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'mean_weighted_transaction_amount_mio'] = 0
    else:
        var_mean = var_amount/var_no_of_trans_no_0
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'mean_weighted_transaction_amount_mio'] = var_mean

In [36]:
# 'total_funding_amount_estimated'
for x in df_urggem_gem_ECAcol_nodoub_wei.index:  
    var_TrackerID = df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'TrackerID']
    df_a = df_urggem_gem_ECAcol_nodoub_wei.copy()
    df_b = df_a[df_a['TrackerID']==var_TrackerID].reset_index(drop=True)
    var_total = 0
    for y in df_b.index:
        if df_b.loc[y, 'weighted_transaction_amount_mio'] == 0:
            var_total += df_b.loc[y, 'mean_weighted_transaction_amount_mio']
        else:
            var_total += df_b.loc[y, 'weighted_transaction_amount_mio']
    df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'total_funding_amount_estimated_mio'] = var_total

In [37]:
# 'weighted_funding_amount_estimated'
for x in df_urggem_gem_ECAcol_nodoub_wei.index:  
    if df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'weighted_transaction_amount_mio'] == 0:
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'weighted_funding_amount_estimated_mio'] = \
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'mean_weighted_transaction_amount_mio']
    else:
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'weighted_funding_amount_estimated_mio'] = \
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'weighted_transaction_amount_mio']

In [38]:
# 'funding_share'
#P: for some units the 'total_funding_amount_estimated' and the 'weighted_funding_amount_estimated' are 0, thus share aso 0.
# in that case split share among the transactions for a unit.
list_IDs_0 = \
set(df_urggem_gem_ECAcol_nodoub_wei[df_urggem_gem_ECAcol_nodoub_wei['total_funding_amount_estimated_mio']==0]['TrackerID'])

for x in df_urggem_gem_ECAcol_nodoub_wei.index: 
    var_ID = df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'TrackerID']
    if var_ID in list_IDs_0:
        df_a = df_urggem_gem_ECAcol_nodoub_wei[df_urggem_gem_ECAcol_nodoub_wei['TrackerID'] == var_ID]
        var_no_units = len(df_a)
        var_share = 1/var_no_units
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'funding_share'] = var_share
    else:
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'funding_share'] = \
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'weighted_funding_amount_estimated_mio']/\
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'total_funding_amount_estimated_mio']

In [39]:
# 'capacity_weighted_by_funding_share'
for x in df_urggem_gem_ECAcol_nodoub_wei.index:
    var_share = df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'funding_share']
    var_capa = df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'Capacity (MW)']
    var_capa_weigh = var_share*var_capa
    if var_capa == 0:
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'capacity_weighted_by_funding_share_MW'] = 0
    else:
        df_urggem_gem_ECAcol_nodoub_wei.loc[x, 'capacity_weighted_by_funding_share_MW'] = var_capa_weigh

In [40]:
#Check the vaules for each new column
for x in ['Amount (in USD 2020)_mio', 'weighted_transaction_amount_mio', 'mean_weighted_transaction_amount_mio', \
          'total_funding_amount_estimated_mio', 'weighted_funding_amount_estimated_mio', \
          'funding_share', 'capacity_weighted_by_funding_share_MW']:
    print(x + ' sum = ' + str(df_urggem_gem_ECAcol_nodoub_wei[x].sum()))
print('number of unique units (should equal sum of funding share) is '+ \
      str(len(set(df_urggem_gem_ECAcol_nodoub_wei['TrackerID']))))
df_a = df_urggem_gem_ECAcol_nodoub_wei.copy()
var_unique_sum = df_a.drop_duplicates(subset = ['TrackerID'])['Capacity (MW)'].sum()
print('capa of uniue units (should equal sum of capacit_weighted) is '+ str(var_unique_sum))

Amount (in USD 2020)_mio sum = 334343.6982629417
weighted_transaction_amount_mio sum = 141026.45288908598
mean_weighted_transaction_amount_mio sum = 163033.7162501834
total_funding_amount_estimated_mio sum = 2118270.767046257
weighted_funding_amount_estimated_mio sum = 163033.7162501834
funding_share sum = 459.0
capacity_weighted_by_funding_share_MW sum = 225484.09999999998
number of unique units (should equal sum of funding share) is 459
capa of uniue units (should equal sum of capacit_weighted) is 225484.1


In [41]:
df_urggem_gem_ECAcol_nodoub_wei.to_csv('.//tables//basis//df_urggem_gem_ECAcol_nodoub_wei_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_20210728.csv', \
                       sep=';', encoding='latin-1', decimal=',')

# 1.6 Provide overview of the data

In [42]:
df_urggem_gem_ECAcol_nodoub_wei = pd.read_csv('.//tables//basis//df_urggem_gem_ECAcol_nodoub_wei_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_20210728.csv', \
                       sep=';', encoding='latin-1', decimal=',', index_col=0)

In [44]:
#creat dicts
df_both = df_urggem_gem_ECAcol_nodoub_wei.copy()
df_state = df_both[df_both['ownership']=='state-owned'].reset_index(drop=True)
df_private = df_both[df_both['ownership']=='private'].reset_index(drop=True)

dict_3_dfs = {'both':df_urggem_gem_ECAcol_nodoub_wei, 'state_owned':df_state, 'private':df_private}

In [45]:
for abc in dict_3_dfs:
    df_abc = dict_3_dfs[abc]
    print('for '+ abc)
    print('DOUBLE COUNTING! from '+ abc +': total amount in USD mio = ' + str(sum(df_abc['Amount (in USD 2020)_mio'])))
    print('from '+ abc +': weighted total amount in USD mio = ' + str(sum(df_abc['weighted_transaction_amount_mio'])))
    print('DOUBLE COUNTING! from '+ abc +' (all matched units): total MW = ' + str(sum(df_abc['Capacity (MW)'])))
    print('from '+ abc +' & GCPT: weighted (matched units): MW by share = ' + str(sum(df_abc['capacity_weighted_by_funding_share_MW'])))
    print('number of unique transactions = ' + str(len(set(df_abc['Transaction Nr']))))
    print('number of distinct units = ' + str(len(set(df_abc['TrackerID']))))
    print('number of distinct plants = ' + str(len(set(df_abc['TrackerLOC']))))
    print('number of unit-transactions = ' + str(len(df_abc)) +'\n')

for both
DOUBLE COUNTING! from both: total amount in USD mio = 334343.69826293935
from both: weighted total amount in USD mio = 141026.45288908592
DOUBLE COUNTING! from both (all matched units): total MW = 1587863.1999999997
from both & GCPT: weighted (matched units): MW by share = 225484.1
number of unique transactions = 429
number of distinct units = 459
number of distinct plants = 204
number of unit-transactions = 2527

for state_owned
DOUBLE COUNTING! from state_owned: total amount in USD mio = 259340.43896294152
from state_owned: weighted total amount in USD mio = 104489.62388908562
DOUBLE COUNTING! from state_owned (all matched units): total MW = 763132.1999999997
from state_owned & GCPT: weighted (matched units): MW by share = 194601.97665384648
number of unique transactions = 367
number of distinct units = 440
number of distinct plants = 191
number of unit-transactions = 1329

for private
DOUBLE COUNTING! from private: total amount in USD mio = 75003.25929999986
from private: w

# 2 Prepare final table for analysis

## 2.1 Create table with all unique units from both datasets

In [46]:
#create 'long' version with all columns
df_unique_units_long = df_both.copy()
df_unique_units_long = df_unique_units_long.drop_duplicates(subset = ['TrackerID']).reset_index(drop=True)

In [48]:
#create a version with few columns only
df_unique_units = df_unique_units_long[['TrackerID', 'TrackerLOC', 'Unit', 'Plant', \
                                   'Country','Sponsor', 'Parent', 'Capacity (MW)','Year']]

## 2.2 Merge with Platts 

### 2.2.1 Check, if unit already manually matched with Platts

In [68]:
df_units_already_matched = \
    pd.read_csv('.//tables//df_units_matched_GCPT_GCFT_urg_Platts19_20210728_nm.csv' ,\
                                      sep=';', encoding='latin-1', decimal=',', index_col=0)
df_units_already_matched.reset_index(drop=True, inplace=True)

In [71]:
# check for each unit, if already in the df_units_already_matched. If not, add!
a = 0
for x in df_unique_units.index:
    var_ID = df_unique_units.loc[x, 'TrackerID']
    if var_ID not in df_units_already_matched['TrackerID'].values:
        a += 1
        var_len = len(df_units_already_matched)
        for y in ['TrackerID', 'TrackerLOC', 'Unit', 'Plant', 'Country', 'Sponsor','Parent', 'Capacity (MW)', 'Year']:
            df_units_already_matched.loc[var_len, y] = df_unique_units.loc[x, y]
print( str(a) + ' not yet matched with platts')

0 not yet matched with platts


In [208]:
df_units_already_matched.to_csv('.//tables//df_units_matched_GCPT_GCFT_urg_Platts_20210323.csv', \
                       sep=';', encoding='latin-1', decimal=',')

### 2.2.2 Get information on companies from Platts

In [81]:
# import table with unique units matched with Platts
df_all_unique_units_matched = \
    pd.read_csv('.//tables//df_units_matched_GCPT_GCFT_urg_Platts19_20210728_nm.csv' ,\
                                      sep=';', encoding='latin-1', decimal=',', index_col=0) #20210323

In [45]:
#TURBMFR Turbine manufacturer or IC engine manufacturer; 
#COMPANY Operator or primary owner of generating unit. 
#SSSMFR Steam supply system (boiler or reactor) manufacturer, N/A for other technologies. 
#GENMFR Generator manufacturer. 
#AE Primary architect/engineering firm. 
#CONSTRUCT Primary construction contractor. 
#PARENT Parent company of plant operator, generally ultimate parent, not immediate parent. \
    #For sorting purposes, the Company has been repeated if there is no Parent identified.
    
#not relevant:
    #PARTMFR Manufacturer of particulate control device for unit. May be N/A. 
    #FGDMFR Manufacturer of FGD scrubber system, if applicable. May be N/A. 
    #NOXMFR Manufacturer of NOX control system, if applicable. May be N/A. 

In [83]:
list_company_types_from_Platts = ['TURBMFR', 'GENMFR', 'PARTMFR', 'FGDMFR', 'NOXMFR', 'SSSMFR',  'AE', 'CONSTRUCT' ]
list_company_types_all = ['Sponsor', 'Parent', \
        'TURBMFR', 'GENMFR', 'PARTMFR', 'FGDMFR', 'NOXMFR', 'SSSMFR',  'AE', 'CONSTRUCT' ]

In [84]:
# get companies from platts, check if either in 2017 or 2019
for x in df_all_unique_units_matched.index:
    if not np.isnan(df_all_unique_units_matched.loc[x, 'UNITID Platts']):
        var_ID = df_all_unique_units_matched.loc[x, 'UNITID Platts']
        if var_ID in list(df_platts_2019['UNITID']):
            for y in df_platts_2019.index:
                if var_ID == df_platts_2019.loc[y, 'UNITID']:
                    for z in list_company_types_from_Platts:
                        df_all_unique_units_matched.loc[x, z] = df_platts_2019.loc[y, z]
        else:
            for y in df_platts_2017.index:
                if var_ID == df_platts_2017.loc[y, 'UNITID']:
                    for z in list_company_types_from_Platts:
                        df_all_unique_units_matched.loc[x, z] = df_platts_2017.loc[y, z]

In [85]:
# if only parent is stated, not sponsor: take sponsor as parent as well:
for x in df_all_unique_units_matched.index:
    if type(df_all_unique_units_matched.loc[x, 'Parent']) != str:
        if type(df_all_unique_units_matched.loc[x, 'Sponsor']) == str:
            df_all_unique_units_matched.loc[x, 'Parent'] = df_all_unique_units_matched.loc[x, 'Sponsor']

In [86]:
#check how many are not nan / empty
df_all_unique_units_matched.notna().sum()

TrackerID              491
TrackerLOC             491
Unit                   491
Plant                  491
Country                491
Sponsor                491
Parent                 491
Capacity (MW)          491
Year                   478
UNIT name Platts       443
PLANT name Platts      443
UNITID Platts          443
Matching Confidence    491
comment                  4
TURBMFR                210
GENMFR                 207
PARTMFR                 45
FGDMFR                  38
NOXMFR                  22
SSSMFR                 202
AE                     260
CONSTRUCT              259
dtype: int64

## 2.3 Add company information from GCFT

In [87]:
# check: Do some units not have information on Platts but in the merged df from the GCFT?
# of course this only works for the plants from GEM GCFT, not urgewald!
# by 'TrackerID' (from GCPT)
# Platts (so df_all_unique_units_matched) : 
list_tpes_platts = ['TURBMFR', 'GENMFR', 'SSSMFR', 'CONSTRUCT']
# GCFT (from the 2019 version. In 2020 no construction and other names!) (so df_unique_units_long) : no 'AE'
list_types_GCFT = ['TurbMFR', 'Equipment (GENMFR)','SSSMFR','Construction']   

In [88]:
dict_comp_types = dict(zip(list_tpes_platts, list_types_GCFT))

In [89]:
# if units have no information from Platts, but from GCFT, add these info
a = 0
for x in df_all_unique_units_matched.index:
    for y in df_unique_units_long.index:
        if df_all_unique_units_matched.loc[x, 'TrackerID'] == df_unique_units_long.loc[y, 'TrackerID']:
            for z in dict_comp_types:
                if type(df_all_unique_units_matched.loc[x, z]) != str:
                    if df_unique_units_long.loc[y, dict_comp_types[z]] != 'TBD':
                        df_all_unique_units_matched.loc[x, z] = df_unique_units_long.loc[y, dict_comp_types[z]]

In [90]:
#check again, if now more information on companies
df_all_unique_units_matched.notna().sum()

TrackerID              491
TrackerLOC             491
Unit                   491
Plant                  491
Country                491
Sponsor                491
Parent                 491
Capacity (MW)          491
Year                   478
UNIT name Platts       443
PLANT name Platts      443
UNITID Platts          443
Matching Confidence    491
comment                  4
TURBMFR                224
GENMFR                 216
PARTMFR                 45
FGDMFR                  38
NOXMFR                  22
SSSMFR                 204
AE                     260
CONSTRUCT              276
dtype: int64

## 2.4 Split companies

often multiple companies are stated -->split!

In [91]:
# rename columns in df: should be with '_1'
for x in list_company_types_all:
    df_all_unique_units_matched[x+'_1'] = df_all_unique_units_matched[x]
    df_all_unique_units_matched.drop(columns=x, inplace=True)

In [92]:
# creat list with the numbers 
list_company_types_all_no = []
for x in list_company_types_all:
    list_company_types_all_no.append(x+'_1')

In [93]:
# sometimes split with "/" (Platts), "," (GCFT, GCPT), ";" (just in case!)

n = 0
m = 0
o = 0

for x in df_all_unique_units_matched.index:
    for y in list_company_types_all:
        var_unsplit_name = df_all_unique_units_matched.loc[x, y+'_1']
        if type(var_unsplit_name)==str:
            if len(var_unsplit_name.split('/')) >1:
                for z in range(0,len(var_unsplit_name.split('/'))):
                    df_all_unique_units_matched.loc[x, y+'_'+str(z+1)] = var_unsplit_name.split('/')[z].strip()
                    if y+'_'+str(z+1) not in list_company_types_all_no:
                        list_company_types_all_no.append(y+'_'+str(z+1))
                n+=1
            elif len(var_unsplit_name.split(',')) >1:
                for z in range(0,len(var_unsplit_name.split(','))):
                    df_all_unique_units_matched.loc[x, y+'_'+str(z+1)] = var_unsplit_name.split(',')[z].strip()
                    if y+'_'+str(z+1) not in list_company_types_all_no:
                        list_company_types_all_no.append(y+'_'+str(z+1))
                m+=1
            elif len(var_unsplit_name.split(';')) >1:
                for z in range(0,len(var_unsplit_name.split(';'))):
                    df_all_unique_units_matched.loc[x, y+'_'+str(z+1)] = var_unsplit_name.split(';')[z].strip()
                    if y+'_'+str(z+1) not in list_company_types_all_no:
                        list_company_types_all_no.append(y+'_'+str(z+1))
                o+=1
print("split by '/' = " +str(n))
print("split by ',' = " +str(m))
print("split by ';' = " +str(o))

not split =101
split by ',' = 258
split by ';' = 5


In [95]:
# 2 times company is just ' ', sometimes '0', or 'TBD' -->replace
for x in df_all_unique_units_matched.index:
    for y in list_company_types_all_no:
        list_to_replace=['', '0', 'TBD']
        for z in list_to_replace:
            if df_all_unique_units_matched.loc[x,y] == z:
                df_all_unique_units_matched.loc[x,y] = np.nan

In [96]:
# if only parent is stated, not sponsor: take sponsor as parent as well: (do it again)
for x in df_all_unique_units_matched.index:
    if type(df_all_unique_units_matched.loc[x, 'Parent_1']) != str:
        if type(df_all_unique_units_matched.loc[x, 'Sponsor_1']) == str:
            df_all_unique_units_matched.loc[x, 'Parent_1'] = df_all_unique_units_matched.loc[x, 'Sponsor_1']

In [97]:
# make list with all split company names
list_all_split_companies = []
for x in list_company_types_all_no:
    for y in df_all_unique_units_matched.index:
        if type(df_all_unique_units_matched.loc[y, x]) == str:
            list_all_split_companies.append(df_all_unique_units_matched.loc[y, x])
list_all_split_companies = list(set(list_all_split_companies))
len(list_all_split_companies)

655

In [98]:
df_all_unique_units_matched.to_csv('.//tables//df_units_matched_GCPT_GCFT_urg_Platts_split_20210728.csv', \
                       sep=';', encoding='latin-1', decimal=',')

## 2.5 Get countries of companies

We used the countries provided in the datasets and complemented them by manually searching for the companies' countries

In [99]:
#import current version with company names with automatically & manually added countries
df_split_abbrev_and_full_names_countries_nm = \
    pd.read_csv('.//tables//df_platts_GCFT_urg_company_split_names_countries_20210324_nm.csv',decimal=',', sep=';', index_col=0)

In [101]:
# create list with all company names that are currently in the df:
list_columns_with_company_names_old = ['ABBREV', 'other_name_1', 'other_name_2', 'other_name_3', 'other_name_4',
       'other_name_5', 'FULLNAME', 'name_clean']
list_company_names_old = []
for x in list_columns_with_company_names_old:
    list_company_names_old.extend(df_split_abbrev_and_full_names_countries_nm[x].values)
list_company_names_old = list(set(list_company_names_old))
len(list_company_names_old)

1260

In [102]:
# check, if there are new companies that are not yet in the list
# print new names and add to list
list_company_names_only_new_ones = []
for x in df_all_unique_units_matched.index:
    for y in list_company_types_all_no:
        var_comp_name = df_all_unique_units_matched.loc[x,y]
        if type(var_comp_name) == str:
            if var_comp_name not in list_company_names_old:
                list_company_names_only_new_ones.append(var_comp_name)
list_company_names_only_new_ones = list(set(list_company_names_only_new_ones))
len(list_company_names_only_new_ones)

20

In [103]:
# add new names from list to df as ABBREV
for x in list_company_names_only_new_ones:
    new_row = {'ABBREV':x}
    df_split_abbrev_and_full_names_countries_nm = df_split_abbrev_and_full_names_countries_nm.append(new_row, ignore_index=True)

In [104]:
def abbrev_to_full_name(abbrev):
    a = abbrev
    for x in df_platts_abbrev.index:
        if df_platts_abbrev.loc[x, 'ABBREV'] == abbrev:
            a = df_platts_abbrev.loc[x, 'FULLNAME']
            break
    return a

In [105]:
# if no fullname given yet, add it. In Platts the country is sometimes stated as well. If so, take it!
for x in df_split_abbrev_and_full_names_countries_nm.index:
    if type(df_split_abbrev_and_full_names_countries_nm.loc[x,'FULLNAME']) != str:
        var_abbrev = df_split_abbrev_and_full_names_countries_nm.loc[x,'ABBREV']
        list_split = abbrev_to_full_name(var_abbrev).split("[")
        if len(list_split) !=1:
            df_split_abbrev_and_full_names_countries_nm.loc[x,'FULLNAME'] = abbrev_to_full_name(var_abbrev)
            df_split_abbrev_and_full_names_countries_nm.loc[x,'COUNTRY'] = list_split[1].replace("]", "").replace(")", "")
        else:
            df_split_abbrev_and_full_names_countries_nm.loc[x,'FULLNAME'] = abbrev_to_full_name(var_abbrev)


In [106]:
# check in table from MA where I manually added countries of developers!
df_dev_HQ_from_MA = pd.read_csv('.//tables//dev_HQ.csv', decimal=',', sep=';', index_col=0)

In [107]:
for x in df_split_abbrev_and_full_names_countries_nm.index:
    if type(df_split_abbrev_and_full_names_countries_nm.loc[x, 'COUNTRY']) != str:
        for y in df_dev_HQ_from_MA.index:
            if df_split_abbrev_and_full_names_countries_nm.loc[x, 'ABBREV'] == df_dev_HQ_from_MA.loc[y, 'DEV']:
                df_split_abbrev_and_full_names_countries_nm.loc[x, 'COUNTRY'] = df_dev_HQ_from_MA.loc[y, 'DEV Country']
            elif df_split_abbrev_and_full_names_countries_nm.loc[x, 'FULLNAME'] == df_dev_HQ_from_MA.loc[y, 'DEV']:
                df_split_abbrev_and_full_names_countries_nm.loc[x, 'COUNTRY'] = df_dev_HQ_from_MA.loc[y, 'DEV Country']

In [109]:
# export this df so I can add the countries manually:
df_split_abbrev_and_full_names_countries_nm.to_csv('.//tables//df_platts_GCFT_urg_company_split_names_countries_20210729.csv', \
    decimal=',', sep=';') 

 ## 2.6 Add countries to dataset with unique units

In [278]:
df_all_unique_units_matched = pd.read_csv('.//tables//df_units_matched_GCPT_GCFT_urg_Platts_split_20210728.csv', \
                       sep=';', encoding='latin-1', decimal=',')

First, import version with manually added countries

In [117]:
#import current version with automatically & manually added countries
df_split_abbrev_and_full_names_countries_nm = \
    pd.read_csv('.//tables//df_platts_GCFT_urg_company_split_names_countries_20210729_nm.csv',decimal=',', sep=';', index_col=0)

In [119]:
#check: how many unique companies (by clean name)?
len(set(df_split_abbrev_and_full_names_countries_nm['name_clean']))

637

In [120]:
# get countries
df_all_unique_units_matched_countries = df_all_unique_units_matched.copy()

In [121]:
# change all company names to df_split_abbrev_and_full_names_countries_nm['name_clean']:
for x in df_all_unique_units_matched_countries.index:
    for y in list_company_types_all_no:
        if type(df_all_unique_units_matched_countries.loc[x,y])==str:
            for z in df_split_abbrev_and_full_names_countries_nm.index:
                for xyz in ['ABBREV', 'other_name_1', 'other_name_2', 'other_name_3', 'other_name_4', 'other_name_5', 'FULLNAME']:
                    if df_all_unique_units_matched_countries.loc[x,y] == df_split_abbrev_and_full_names_countries_nm.loc[z,xyz]:
                        df_all_unique_units_matched_countries.loc[x,y] = df_split_abbrev_and_full_names_countries_nm.loc[z,'name_clean']
                        df_all_unique_units_matched_countries.loc[x,y+'_Country'] = df_split_abbrev_and_full_names_countries_nm.loc[z,'COUNTRY']

In [122]:
df_all_unique_units_matched_countries.to_csv\
    ('.//tables//df_unique_units_countries_plants_finance_merge_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_platts_'+tz+'_20210729.csv' ,\
                                      sep=';', encoding='latin-1', decimal=',')

## 2.7 Finalize and export dataset with unique units

In [320]:
df_all_unique_units_matched_countries = pd.read_csv\
    ('.//tables//df_unique_units_countries_plants_finance_merge_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_platts_'+tz+'_20210729.csv' ,\
                                      sep=';', encoding='latin-1', decimal=',', index_col=0)

In [129]:
# some IDs in df_all_unique_units_matched_countries_short are from older versions. Drop:
df_only_matched_units = df_all_unique_units_matched_countries.copy()

for x in df_only_matched_units.index:
    if df_only_matched_units.loc[x, 'TrackerID'] not in df_both['TrackerID'].values:
        df_only_matched_units.drop(x, inplace=True)
len(df_only_matched_units)

459

In [131]:
#change the order of columns
df_only_matched_units_order = df_only_matched_units[['TrackerID', 'TrackerLOC', 'Unit', 'Plant', 'Country', 'Capacity (MW)',\
       'Year', 'UNIT name Platts', 'PLANT name Platts', 'UNITID Platts',\
       'Matching Confidence', 'Sponsor_1', 'Sponsor_1_Country', \
        'Sponsor_2', 'Sponsor_2_Country', 'Sponsor_3', 'Sponsor_3_Country', 'Sponsor_4', 'Sponsor_4_Country', \
        'Parent_1', 'Parent_1_Country', 'Parent_2', 'Parent_2_Country', 'Parent_3', 'Parent_3_Country', \
        'Parent_4', 'Parent_4_Country', 'Parent_5', 'Parent_5_Country', 'Parent_6', 'Parent_6_Country', \
        'TURBMFR_1', 'TURBMFR_1_Country', 'TURBMFR_2', 'TURBMFR_2_Country', 'GENMFR_1', 'GENMFR_1_Country', \
        'GENMFR_2', 'GENMFR_2_Country', 'PARTMFR_1','PARTMFR_1_Country', \
       'FGDMFR_1', 'FGDMFR_1_Country', 'NOXMFR_1', 'NOXMFR_1_Country', 'SSSMFR_1', 'SSSMFR_1_Country', \
        'AE_1',  'AE_1_Country', 'AE_2', 'AE_2_Country', 'AE_3', 'AE_3_Country', 'CONSTRUCT_1','CONSTRUCT_1_Country', \
        'CONSTRUCT_2', 'CONSTRUCT_2_Country', 'CONSTRUCT_3',  'CONSTRUCT_3_Country', 'CONSTRUCT_4',  'CONSTRUCT_4_Country']]

In [326]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
 #    print(df_only_matched_units_order.notna().sum())

In [132]:
df_only_matched_units_order.to_csv\
    ('.//tables//df_only_matched_units_order_countries_plants_finance_merge_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_platts_'+tz+'_20210729.csv' ,\
                                      sep=';', encoding='latin-1', decimal=',')

## 2.8 Add companies and countries to the merged unit-transaction dataset

In [133]:
df_both = df_urggem_gem_ECAcol_nodoub_wei.copy()

In [134]:
df_only_matched_units_order = pd.read_csv\
    ('.//tables//df_only_matched_units_order_countries_plants_finance_merge_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_platts_'+tz+'_20210729.csv' ,\
                                      sep=';', encoding='latin-1', decimal=',', index_col=0)

In [135]:
# drop some columns that are already in the merged unit-transaction df
df_only_matched_units_order_short = df_only_matched_units_order.copy()
list_drop_a = ['TrackerLOC', 'Unit', 'Plant', 'Country', 'Capacity (MW)', 'Year']

for x in list_drop_a:
    df_only_matched_units_order_short.drop(columns=x, inplace=True)

In [136]:
# drop some columns in df_both before merging the two:
list_drop_b = ['Financial Close','Region.fin','Recipient Subregion','Construction', 'EPC Company', \
             'Equipment (GENMFR)', 'TurbMFR', 'Boiler Maker', 'Sector Group', 'Sector', 'Status.fin', 'SSSMFR', \
       'OpYEAR', 'Latitude.fin', 'Longitude.fin', 'Location Accuracy', 'Subnational unit (province, state)',\
            'Sponsor', 'Parent']
for x in list_drop_b:
    df_both.drop(columns=x,inplace=True)

In [137]:
df_both_platts = pd.merge(df_both, df_only_matched_units_order_short, on='TrackerID', how='outer')

In [139]:
df_both_platts.to_csv('df_unit_transactions_merged_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_platts_'+tz+'_20210729.csv', \
                                      sep=';', encoding='latin-1', decimal=',')

## 2.9 change order of columns

In [ ]:
df_both_platts = pd.read_csv('df_unit_transactions_merged_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_platts_'+tz+'_20210729.csv', \
                                      sep=';', encoding='latin-1', decimal=',', index_col = 0)


In [141]:
#change order of columns
list_columns_all = ['Transaction Nr', 'ownership', 'data','Financing Country', 'Institution', 'Type short', 'Project Name', \
        'matching confidence', 'Comment', 'MW', 'Deflation_Factor_2020', \
        'Amount (in USD 2020)_mio', 'Amount (in USD)', 'Year of Close', 'Recipient Country', 'Phase ID', \
        'number_of_units_matched', 'number_of_trans_matched', 'weighted_transaction_amount_mio', \
       'mean_weighted_transaction_amount_mio', 'total_funding_amount_estimated_mio', 'funding_share', \
       'capacity_weighted_by_funding_share_MW','weighted_funding_amount_estimated_mio', \
                'TrackerID', 'TrackerLOC', \
        'Country', 'Unit', 'Plant', 'Chinese Name', 'Other names', 'Capacity (MW)', 'Status',  \
       'Year', 'RETIRED', 'Plant age', 'Planned Retire', 'Combustion technology', 'Coal type', \
       'Coal source', 'Location','Heat rate', 'Emission factor', \
       'Annual CO2 (million tonnes / annum)', \
       'Lifetime CO2', \
       'UNIT name Platts', 'PLANT name Platts', 'UNITID Platts', 'Matching Confidence', \
           'Sponsor_1', 'Sponsor_1_Country', 'Sponsor_2', 'Sponsor_2_Country', 'Sponsor_3', 'Sponsor_3_Country', \
        'Sponsor_4', 'Sponsor_4_Country', \
        'Parent_1', 'Parent_1_Country', 'Parent_2', 'Parent_2_Country', 'Parent_3', 'Parent_3_Country', \
        'Parent_4', 'Parent_4_Country', 'Parent_5', 'Parent_5_Country', 'Parent_6', 'Parent_6_Country', \
        'Insurer_1', 'Insurer_1_Country', 'Insurer_2', 'Insurer_2_Country', \
        'TURBMFR_1', 'TURBMFR_1_Country', 'TURBMFR_2', 'TURBMFR_2_Country', 'GENMFR_1', 'GENMFR_1_Country', \
        'GENMFR_2', 'GENMFR_2_Country', 'PARTMFR_1','PARTMFR_1_Country', \
       'FGDMFR_1', 'FGDMFR_1_Country', 'NOXMFR_1', 'NOXMFR_1_Country', 'SSSMFR_1', 'SSSMFR_1_Country', \
        'AE_1',  'AE_1_Country', 'AE_2', 'AE_2_Country', 'AE_3', 'AE_3_Country', 'CONSTRUCT_1','CONSTRUCT_1_Country', \
        'CONSTRUCT_2', 'CONSTRUCT_2_Country', 'CONSTRUCT_3',  'CONSTRUCT_3_Country', 'CONSTRUCT_4',  'CONSTRUCT_4_Country']

df_both_platts_order = df_both_platts[list_columns_all]


# 2.10 Export final unit-transaction dataset

In [ ]:
df_both_platts_order.to_csv('df_order_unit_transactions_merged_GCPT_'+pz+'_GCFT_'+fz+'_urg_'+uz+'_platts_'+tz+'_20210729.csv', \
                                      sep=';', encoding='latin-1', decimal=',')